# Census Data Decision Tree using ID3 algroithm and Information Gain

In [1]:
import math
import pandas as pd

In [2]:
#labeledPositive = number of positive data points, totalSize = total number of instances in data set
#note this function will only work for data sets with a binary class label (true/false, >=50k/<50k, etc)
#also "positive" and "negative" are interchangable - entropy will calc the same for either.
#Returns entropy value
def calc_entropy(labeledPositive,totalSize):
    labeledNegative = totalSize-labeledPositive
    #if one of our variables is 0, entropy is 0. Special case to avoid divide by zero errors
    if labeledNegative == 0 or labeledPositive == 0:
        return 0
    #else return entropy calculation
    else:
        return -((labeledPositive/totalSize)*math.log((labeledPositive/totalSize),2)+(labeledNegative/totalSize)*math.log((labeledNegative/totalSize),2))

In [3]:
#returns Information Gain value given a list of feature names and a dataframe containing 2 columns - the data column in question and the class label
def calc_IG(values,data):
    totalIG=0
    #find names of feature and class label
    names = []
    for col in data.columns:
        names.append(col)
    classLabel = data[names[1]].unique()
    #for every unique value in data, add to IG
    for v in values:     
        #isolate just data with the unique attribute we care about
        tempdf = data[data[names[0]] == v]
        numRows = len(tempdf.index)
        #find all values with a certain class label
        numOccur = len(tempdf[tempdf[names[1]] == classLabel[0]])
        #add to totalIG using information gain calculation
        totalIG += ((numRows/len(data.index))*(calc_entropy(numOccur,numRows)))
    numOccur = len(data[data[names[1]] == classLabel[0]])
    #finally, add total IG and subtract the calculations we did above in for loop per IG calculation
    totalIG = calc_entropy(numOccur,len(data.index)) - totalIG    
    return totalIG

In [4]:
#returns a string with the name of the feature with highest IG
def chooseFeature(data):
    #setup class label
    classLabel = data[data.columns[-1]]
    data = data.drop([data.columns[-1]], axis=1)
    bestIG=0
    bestIGname=data.columns[0]
    
    #for each column, calculate IG. If it is more than bestIG, replace bestIG. 
    for col in data:
        #attach one column of data with class label
        toCalc = pd.concat([data[col], classLabel], axis=1, sort=False)
        #setup column names
        name=[]
        for c in toCalc.columns:
            name.append(c)
        #send values to calc_IG function to calculate
        tempIG=calc_IG(data[col].unique(),toCalc)
        #if better or equal, replace
        if tempIG >= bestIG:
            bestIG = tempIG
            bestIGname = name[0]
    return bestIGname

In [5]:
#returns dictionary of dictionaries given data and a list of attributes to test
def build_tree(data,attr,fulldata):
    #setup tree, header names, and class label for use later
    tree = {}
    name=[]
    for c in data.columns:
        name.append(c)
    classLabel = data[data.columns[-1]]
    classLabelName = name[-1]
    del name[-1]
    
    if (data.empty):
        #exit case dataset is empty, we are done
        return (fulldata[fulldata.columns[-1]].value_counts()[:1].index[0])
        
        
    elif (len(classLabel.unique())==1):
        #exit case for all examples positive/negative
        
        #return the value of class label 
        return classLabel.unique().tolist()[0]    
    
    elif len(data.columns.values)==1:
        #exit case for tested all attributes
        return data.iloc[0][0]
    
    else:
        #else, continue recursion
        #select best feature and remove it from list of attributes
        featureName = chooseFeature(data)
        #get all possible values of target feature
        target_vals = fulldata[featureName].unique()
        #the output of this for loop will be put into a new dictionary, which will become the "value" to featureName in tree
        temptree = {}
        #recurse for each unique value of selected feature with only those rows included
        for value in target_vals:
            #select only rows with each value of target feature
            tempdata = data.loc[data[featureName] == value]
            #drop featureName column data as we have already used it
            tempdata = tempdata.drop([featureName],axis=1)
            #set   value : {recursively build new tree}
            temptree[value] = build_tree(tempdata,attr,fulldata)
        
        #add to tree
        tree[featureName] = temptree
        return tree

In [6]:
#calculates and prints accuracy of test data given data and a tree (dictionary of dictionaries)
def predict(testData,tree):
    #setup testData to be processed
    name=[]
    for c in testData.columns:
        name.append(c)
    classLabel = testData[testData.columns[-1]]
    classLabelName = name[-1]
    del name[-1]
    
    prediction=[]
    #for each row in testData:
    for i in range(len(testData)):
        #add prediction for row i to list
        guess=predict_row(testData.loc[i],tree)
        #had 1 or 2 instances where guess was None out of the dataset of 15000, not sure why. 
        #this if statement simply adds an incorrect guess if somehow predict_row returns none.
        if guess is None:
            guess="N/A"
        #had some errors with list/non-list values - if guess is returned in list form, append its value to prediction
        if type(guess) is list:
            prediction.append(guess[0])
        #else, simply append guess
        else:
            prediction.append(guess)
    
    numCorrect=0
    #count correct. This could easily be combined into one for loop but I seperated them for clarity.    
    for i in range(len(testData)):
        if (prediction[i] == classLabel[i]):
            numCorrect = numCorrect+1
    
    #print(">50K:",prediction.count('>50K'),"\n<=50K:",prediction.count('<=50K'))
    #print information
    print("********************\n\n")
    print("Number of testing examples:",len(testData))
    print("Number correctly predicted:",numCorrect)
    print("Number incorrectly predicted:",(len(testData)-numCorrect))
    print("Accuracy:",(numCorrect/len(testData))*100,"%")
    print("\n\n********************")

In [7]:
#returns string with prediction for a given row based on tree
def predict_row(row,tree):
    for key in tree:
        #Find the inner dictionary that matches with the key that the tree is looking for. In the inner dictionary,
        #set newTree to be the dictionary or value corresponding to the value that 'row' has.
        #this line is very confusing, I tried to explain it as clearly as possible  
        newTree = tree[key][row[key]]
        #if value is also a dictionary, recurse
        if isinstance(newTree,dict): 
            return predict_row(row,newTree)
        #if newtree's NOT a dictionary: return value
        else:
            return newTree

In [8]:
df = pd.read_csv("assets/census_training.csv")
dfTest = pd.read_csv("assets/census_training_test.csv")

#for testing purposes
#df = pd.read_csv("assets/playtennis.csv")
#dfTest = pd.read_csv("assets/playtennis.csv")

#setup for build_tree function
name = []
for c in df.columns:
    name.append(c)
del name[-1]



#run prediction
predict(dfTest,build_tree(df,name,df))

********************


Number of testing examples: 15028
Number correctly predicted: 11759
Number incorrectly predicted: 3269
Accuracy: 78.24727175938249 %


********************
